In [23]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive

# Team Casimir Funk Dietary Reference Intakes Notebook
In this notebook, we create the our dietary reference intakes functions.

In [25]:
def handle_query(age, sex):
    '''
    Description
    --------------------------------------------------
    This is a function to handle the Age & Sex Query 
    for our user.
    
    Inputs
    --------------------------------------------------
    + age : string, integer, or float; must be able to 
        converted to an integer
    + sex : string; either 'Male' or 'Female'
    
    Outputs
    --------------------------------------------------
    + key : string; key that corresponds to a column 
            in us_diet_minimums.csv
    + diet_mins : pandas dataframe; holds the data 
            from us_diet_minimums.csv
    '''
    age = int(age)
    sex = sex.title()
    diet_mins = pd.read_csv('./data/us_diet_minimums.csv')
    diet_mins.columns
    
    if (sex == 'Male' or sex == 'Female') and (age > 0):
        s = sex[0]
        age_cols = diet_mins.columns[2:]
        ages = [ent.split(" ")[1].replace("+", "") for ent in age_cols]
        ages = [ages[i] for i in range(len(ages)) if ages[i - 1] != ages[i]] # Removes duplicates
        ages_min = np.array([int(age.split("-")[0]) for age in ages])

        age_range = ages[sum(ages_min <= age) - 1]
        if age_range == '1-3':
            s = 'C'
        if age_range == '51':
            age_range += '+'

        key = f'{s} {age_range}'
        return key, diet_mins
    else:
        return -29, diet_mins # -29 is error code for invalid inputs

In [26]:
def get_drefs(key, diet_mins):
    '''
    Description
    --------------------------------------------------
    This is a function that returns the Dietary 
    References from us_diet_minimums.csv for the 
    user's desired age and sex.
    
    Inputs
    --------------------------------------------------
    + key : string; key that corresponds to a column 
            in us_diet_minimums.csv
    + diet_mins : pandas dataframe; holds the data 
            from us_diet_minimums.csv
    
    Outputs
    --------------------------------------------------
    + drefs : pandas dataframe; dietary reference 
            intakes for the user's desired age and 
            sex
    '''
    if key != -29:
        drefs = diet_mins[['Nutrition', key]].set_index('Nutrition')
        return drefs
    else:
        return -29 # -29 is error code for invalid inputs

In [27]:
def dref_wrapper(Age, Sex, Show = False):
    '''
    Description
    --------------------------------------------------
    This is a wrapper function that combines the
    functionality of handle_query() & get_drefs().

    Inputs
    --------------------------------------------------
    + Age : string, integer, or float; must be able to 
        converted to an integer
    + Sex : string; either 'Male' or 'Female'
    + Show : boolean; indicates whether drefs 
            should be displayed. Important for widgets
    
    Outputs
    --------------------------------------------------
    + drefs : pandas dataframe; dietary reference 
            intakes for the user's desired age and 
            sex
    '''
    key, diet_mins = handle_query(Age, Sex)
    drefs = get_drefs(key, diet_mins)
    if type(drefs) != int:
        # So that drefs can be displayed in widgets
        if Show:
            display(drefs)
        return drefs
    else:
        print('Enter a Valid Age and Sex')
        return -29 # -29 is error code for invalid inputs

In [28]:
def compare_groups(groups, show = True):
    '''
    Description
    --------------------------------------------------
    This is a function that allows the user to compare
    the dietary reference intakes for different 
    groups.

    Inputs
    --------------------------------------------------
    + groups : list; (age, sex) tuples where age
            is a value that can be converted to 
            an integer and sex is a string that is 
            either Male or Female
    + show : boolean; indicates whether comp_df 
        should be displayed. Important for widgets
    
    Outputs
    --------------------------------------------------
    + comp_df : pandas dataframe; dietary reference 
            intakes for the user's desired groups
    '''
    # Empty dataframe that we will add our groups to compare
    comp_df = pd.DataFrame()
    valid_groups = True
    
    for group in groups:
        drefs = dref_wrapper(group[0], group[1])
        if type(drefs) == int:
            print(f'Invalid Group: Age = {group[0]}; Sex = {group[1]}')
            valid_groups = False
            break
        else:
            col = drefs.columns[0]
            comp_df[col] = drefs[col]
    
    if valid_groups: # Only returns comp_df if all groups valid
        return comp_df
    else:
        return -29 # -29 is error code for invalid inputs

In [29]:
def interactive_dref():
    '''
    Description
    --------------------------------------------------
    This is a function that makes dref_wrapper() 
    interactive.

    Inputs
    --------------------------------------------------
    + None
    
    Outputs
    --------------------------------------------------
    + Interactive widget is displayed
    '''
    widget = interactive(dref_wrapper, Age = (1, 120, 1), Sex = ['Male', 'Female'], Show = True)
    display(widget)

In [36]:
def interactive_comp_helper(Age1, Sex1, Age2, Sex2):
    '''
    Description
    --------------------------------------------------
    This is a function that is a modified version of
    compare_groups() that only allows the user to
    compare two groups. This is the function that is
    implemented in interactive_comp().

    Inputs
    --------------------------------------------------
    + None
    
    Outputs
    --------------------------------------------------
    + comp_df is displayed
    '''
    groups = [(Age1, Sex1), (Age2, Sex2)]
    comp_df = compare_groups(groups, show = True)
    display(comp_df)

def interactive_comp():
    '''
    Description
    --------------------------------------------------
    This is a function that makes compare_groups()
    interactive.

    Inputs
    --------------------------------------------------
    + None
    
    Outputs
    --------------------------------------------------
    + Interactive widget is displayed
    '''
    widget = interactive(interactive_comp_helper, Age1 = (1, 120, 1), Sex1 = ['Male', 'Female'],
                    Age2 = (1, 120, 1), Sex2 = ['Male', 'Female'])
    display(widget)

In [38]:
def dref_choose_mode(Mode):
    '''
    Description
    --------------------------------------------------
    This is a function that allows the user to choose
    whether they want to use interactive_dref() or 
    interactive_comp().

    Inputs
    --------------------------------------------------
    + Mode : string; Either 'Get Dietary Reference 
    Intakes' or 'Compare Groups'
    
    Outputs
    --------------------------------------------------
    + comp_df is displayed
    '''
    if Mode == 'Compare Groups':
        interactive_comp()
    else:
        interactive_dref()

def dref_interactive_wrapper():
    '''
    Description
    --------------------------------------------------
    This is a function that makes dref_choose_mode()
    interactive.

    Inputs
    --------------------------------------------------
    + None
    
    Outputs
    --------------------------------------------------
    + Interactive widget is displayed
    '''
    widget = interactive(dref_choose_mode, Mode = ['Get Dietary Reference Intakes',
                                                            'Compare Groups'])
    display(widget)